<a href="https://colab.research.google.com/github/vggls/M.Sc._AI_Thesis/blob/main/experiments/breakhis_resnet50/HAAS_Results.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **1. Imports**

In [ ]:
pip install grad-cam

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 36.2 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for grad-cam: filename=grad_cam-1.4.6-py3-none-any.whl size=38261 sha256=f6e526dd9ba8add17c86de85b0b6290418a91e0739af1dd29b6ced92160e1aba
  Stored in directory: /root/.cache/pip/wheels/53/02/43/1f75726b5c28459596067ad91e36951463c01273eef661f09f
Successfully built grad-cam


In [ ]:
import torch
from torchvision  import datasets, transforms
from pytorch_grad_cam import GradCAM, HiResCAM

In [ ]:
# custom written code
from breakhis_slide_allocation import allocate_slides_to_datasets
from breakhis_binary import create_datasets, summaries
from haas import Haas, HA_image

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# **2. Download from Kaggle**

In [ ]:
! mkdir ~/.kaggle
!cp /content/drive/MyDrive/kaggle.json ~/.kaggle/kaggle.json

In [ ]:
! kaggle datasets download ambarish/breakhis

100% 3.98G/3.99G [01:59<00:00, 40.2MB/s]
100% 3.99G/3.99G [02:00<00:00, 35.7MB/s]


In [ ]:
! unzip breakhis.zip

# **3. Test Dataset**

In [ ]:
import shutil
shutil.rmtree('./test_dataset')

In [ ]:
train_transforms = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(p=0.2),
    transforms.RandomVerticalFlip(p=0.2),
    transforms.RandomRotation(20),
    transforms.ToTensor(),
    transforms.Normalize(torch.Tensor([0.5, 0.5, 0.5]), torch.Tensor([0.5, 0.5, 0.5]))
])

# to be applied to validation and test data
test_transforms = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(torch.Tensor([0.5, 0.5, 0.5]), torch.Tensor([0.5, 0.5, 0.5]))
])

In [ ]:
train_dataset, validation_dataset, test_dataset= create_datasets(train_transforms = train_transforms, 
                                                                 test_transforms = test_transforms)

No benign slides overlap in training, validation and test sets!
No malignant slides overlap in training, validation and test sets!


In [ ]:
# do not need them in this notebook
import shutil
shutil.rmtree('./training_dataset')
shutil.rmtree('./validation_dataset')

# **4. ResNet50**

In [ ]:
drive_path = './drive/MyDrive/Colab_Notebooks/dataset_models/BreakHis/Models/'
resnet50 = torch.load(drive_path + 'breakhis_resnet50.pt')
resnet50 = resnet50.cuda()
resnet50 = resnet50.eval()

In [ ]:
all_layers = [resnet50.conv1, resnet50.bn1, resnet50.layer1, resnet50.layer2, resnet50.layer3, resnet50.layer4]
for child in all_layers:
    for param in child.parameters():
        param.requires_grad = True

In [ ]:
resnet50_gradcam_instance = GradCAM(model=resnet50, target_layers=[resnet50.layer4[2].conv3], use_cuda=True)
resnet50_hirescam_instance = HiResCAM(model=resnet50, target_layers=[resnet50.layer4[2].conv3], use_cuda=True)

print('-->GradCAM')
gradcam_score, _, _, _ = Haas(test_dataset, resnet50, resnet50_gradcam_instance)
print('HAAS score:', gradcam_score)
print('---------------------------')
print('-->HiResCAM')
hirescam_score, _, _, _ = Haas(test_dataset, resnet50, resnet50_hirescam_instance)
print('HAAS score:', hirescam_score)

-->GradCAM
No. of test images:  1557
Total time: 112.51391005516052 secs
Accuracy over original images:  0.845
Accuracy over HA images:  0.783
HAAS score: 0.927
---------------------------
-->HiResCAM
No. of test images:  1557
Total time: 106.97418308258057 secs
Accuracy over original images:  0.845
Accuracy over HA images:  0.791
HAAS score: 0.936
